In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import csv
import json

In [2]:
TEST_COLS = ['app_id', 'author_id', 'content',]
GAMES_COLS = ['app_id', 'name', 'release_date', 'is_free', 
              'price_overview',]
            #   'languages']
REVIEWS_COLS = [
    'app_id', 'review_score', 'positive', 'negative', 'recommendations',
    'steamspy_positive', 'steamspy_negative'
]

In [3]:
test = pd.read_csv(
    '../../data/games/test.csv',
    usecols=TEST_COLS,
    dtype={'id': int, 'app_id': int, 'author_id': int,},
    low_memory=False
)
test.fillna(' ', inplace=True)
test['content'] = test['content'].astype(str)
print(test.isna().sum())
print('test:', test.shape)
test_app_ids = set(test['app_id'].unique())


replaced_app_ids = {
    80:   test.index[test['app_id'] == 80].tolist(),
    1256: test.index[test['app_id'] == 1256].tolist(),
    1257: test.index[test['app_id'] == 1257].tolist(),
    1258: test.index[test['app_id'] == 1258].tolist(),
}

test.loc[test['app_id'] == 80, 'app_id'] = 100
test.loc[test['app_id'] == 1256, 'app_id'] = 1250
test.loc[test['app_id'] == 1257, 'app_id'] = 1250
test.loc[test['app_id'] == 1258, 'app_id'] = 1250


games = pd.read_csv(
    '../../data/games/games.csv',
    usecols=GAMES_COLS,
    dtype={'app_id': int, 'price_overview': str},
    engine='python', 
    sep=',',
    quotechar='"',
    escapechar='\\',  
    on_bad_lines='warn' 
)

games.loc[games['app_id'] == 80, 'app_id'] = 100

games = games[games['app_id'].isin(test_app_ids)]
print('games:', games.shape)
def parse_price(po):
    try:
        return json.loads(po).get('final', np.nan)
    except:
        return np.nan

games['price'] = games['price_overview'].apply(parse_price)



reviews = pd.read_csv(
    '../../data/games/reviews.csv',
    usecols=REVIEWS_COLS,
    na_values='\\N',
    keep_default_na=True,
    engine='python',
    sep=',',
    quotechar='"',
    escapechar='\\',
    on_bad_lines='warn'
)
reviews.loc[reviews['app_id'] == 80, 'app_id'] = 100

num_cols = [
    'review_score','positive','negative',
    'recommendations','steamspy_positive','steamspy_negative'
]


for col in num_cols:
    reviews[col] = pd.to_numeric(reviews[col], errors='coerce')


reviews = reviews[reviews['app_id'].isin(test_app_ids)]
print('reviews:', reviews.shape)

print(test.shape)


app_id       0
content      0
author_id    0
dtype: int64
test: (60346, 3)
games: (47, 5)
reviews: (47, 7)
(60346, 3)


In [4]:
print(games.iloc[1]['price_overview'])

{"final": 499, "initial": 499, "currency": "EUR", "final_formatted": "4,99€", "discount_percent": 0, "initial_formatted": ""}


In [5]:
def parse_price_fields(po):
    try:
        obj = json.loads(po)
    except Exception:
        return pd.Series({
            'price_final':       np.nan,
            'price_initial':     np.nan,
            'discount_percent':  np.nan,
            'currency':          None
        })
    return pd.Series({
        'price_final':       obj.get('final', np.nan),
        'price_initial':     obj.get('initial', np.nan),
        'discount_percent':  obj.get('discount_percent', np.nan),
        'currency':          obj.get('currency')
    })
    
price_df = games['price_overview'].apply(parse_price_fields)
games = pd.concat([games, price_df], axis=1)
games.drop(columns=['price_overview'], inplace=True)

print(games.shape)

(47, 9)


In [6]:
games.isna().sum()
print(games.head(45))

    app_id                                 name release_date  is_free  \
0       10                       Counter-Strike   2000-11-01        0   
1       20                Team Fortress Classic   1999-04-01        0   
2       30                        Day of Defeat   2003-05-01        0   
3       40                   Deathmatch Classic   2001-06-01        0   
4       50            Half-Life: Opposing Force   1999-11-01        0   
5       60                             Ricochet   2000-11-01        0   
6       70                            Half-Life   1998-11-08        0   
7      100       Counter-Strike: Condition Zero   2004-03-01        0   
8      130                Half-Life: Blue Shift   2001-06-01        0   
10     220                          Half-Life 2   2004-11-16        0   
11     240               Counter-Strike: Source   2004-11-01        0   
12     280                    Half-Life: Source   2004-06-01        0   
13     300                Day of Defeat: Source   2

In [7]:

games.loc[games['is_free'] == 1, ['price_final', 'price_initial', 'discount_percent']] = 0

median_price = games['price_final'].median()
games['price_final']   = games['price_final'].fillna(median_price)
games['price_initial'] = games['price_initial'].fillna(games['price_final'])
games['discount_percent'] = games['discount_percent'].fillna(0)

games['price'] = games['price'].fillna(median_price)

most_common_currency = games['currency'].mode().iloc[0]
games['currency'] = games['currency'].fillna(most_common_currency)

print(games.isna().sum())

app_id              0
name                0
release_date        0
is_free             0
price               0
price_final         0
price_initial       0
discount_percent    0
currency            0
dtype: int64


In [8]:
games['release_date'] = pd.to_datetime(games['release_date'], errors='coerce')
games['release_year'] = games['release_date'].dt.year

games = games.sort_values('app_id')
games['release_year'] = games['release_year'].interpolate(method='linear')
games['release_year'] = games['release_year'].round().astype('Int64')
games['release_year'] = (
    games['release_year']
    .fillna(method='bfill')
    .fillna(method='ffill')
    .astype(int)
)
games = games.sort_index()

games.drop(columns=['release_date'], inplace=True)
print(games.head(15))

    app_id                            name  is_free    price  price_final  \
0       10                  Counter-Strike        0    819.0        819.0   
1       20           Team Fortress Classic        0    499.0        499.0   
2       30                   Day of Defeat        0    499.0        499.0   
3       40              Deathmatch Classic        0    499.0        499.0   
4       50       Half-Life: Opposing Force        0    499.0        499.0   
5       60                        Ricochet        0    499.0        499.0   
6       70                       Half-Life        0  11399.0      11399.0   
7      100  Counter-Strike: Condition Zero        0    819.0        819.0   
8      130           Half-Life: Blue Shift        0    499.0        499.0   
10     220                     Half-Life 2        0  38500.0      38500.0   
11     240          Counter-Strike: Source        0    975.0        975.0   
12     280               Half-Life: Source        0    975.0        975.0   

In [9]:
reviews.head(15)

,app_id,review_score,positive,negative,recommendations,steamspy_positive,steamspy_negative
0,10,9.0,235403.0,6207.0,153259.0,235397.0,6207.0
1,20,8.0,7315.0,1094.0,6268.0,7314.0,1092.0
2,30,8.0,6249.0,672.0,4146.0,6246.0,672.0
3,40,8.0,2542.0,524.0,2218.0,2541.0,525.0
4,50,9.0,22263.0,1111.0,20144.0,22260.0,1112.0
5,60,8.0,4791.0,1013.0,4145.0,4790.0,1013.0
6,70,9.0,129364.0,4652.0,95770.0,129338.0,4660.0
7,100,8.0,23316.0,2220.0,18898.0,23316.0,2218.0
8,130,8.0,15859.0,1330.0,14830.0,15857.0,1330.0
10,220,9.0,196153.0,4851.0,159918.0,196156.0,4847.0


In [10]:
rev_agg = reviews.groupby('app_id').agg({
    'review_score':       'mean',
    'positive':           'sum',
    'negative':           'sum',
    # 'metacritic_score':   'mean',
    'recommendations':    'sum',
    'steamspy_positive':  'sum',
    'steamspy_negative':  'sum'
}).reset_index()

print('rev_agg:', rev_agg.shape)

rev_agg: (47, 7)


In [11]:
print(test.shape)
test_ext = test.merge(
    rev_agg,
    how='left',
    on='app_id'
)
print(test_ext.shape)

(60346, 3)
(60346, 9)


In [12]:
for app_id in test_ext['app_id'].unique():
    if app_id not in games['app_id'].values:
        print(f'App ID {app_id} not found in games data.')
        
        
count_1230 = (test_ext['app_id'] == 1230).sum()
count_1280 = (test_ext['app_id'] == 1280).sum()
print(f' {count_1230}')
print(f'{count_1280}')
    

 28
76


In [13]:
# test_ext = test_ext[~test_ext['app_id'].isin([1230, 1280])]
print(test_ext.shape)

(60346, 9)


In [14]:
for app_id in test_ext['app_id'].unique():
    if app_id not in games['app_id'].values:
        print(f'App ID {app_id} not found in games data.')
for app_id in test_ext['app_id'].unique():
    if app_id not in reviews['app_id'].values:
        print(f'App ID {app_id} not found in reviews data.')

In [15]:
test_ext = test_ext.merge(
    games,
    how='left',
    on='app_id'
)
print(test_ext.shape)

(60346, 17)


In [16]:
print(games.shape)
print(test_ext.isna().sum())

(47, 9)
app_id               0
content              0
author_id            0
review_score         0
positive             0
negative             0
recommendations      0
steamspy_positive    0
steamspy_negative    0
name                 0
is_free              0
price                0
price_final          0
price_initial        0
discount_percent     0
currency             0
release_year         0
dtype: int64


In [17]:
for original_id, idx_list in replaced_app_ids.items():
    test.loc[idx_list, 'app_id'] = original_id

In [18]:
test_ext.to_csv('../../data/games/test_ext.csv', index=False)